In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [25]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    
    #solve for the duration in minutes and return to the df

    df['duration']=(df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])

    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    #drop rows
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    #Feature Selection
    categorical=['PULocationID','DOLocationID']
    numerical=['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [29]:
df_train=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_val=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [31]:
categorical = ['PULocationID','DOLocationID'] 

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [32]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.81183265470218

In [37]:
with open('Chidiv47R/workspaces/MLOps-Activity/blob/main/02-MLregister/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

FileNotFoundError: [Errno 2] No such file or directory: 'workspaces/MLOps-Activity/blob/main/02-MLregister/lin_reg.bin'